In [26]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import itertools
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings('ignore')

## Neural Networks

- Loading data as txt - renamed the neuralware file to upload

In [9]:
Faults = pd.read_table('/Users/jennyskytta/Downloads/Faults.txt', header=None, names = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter", "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300", "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index", "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index", "Luminosity_Index", "SigmoidOfAreas", "Pastry", "Z_Scratch", "K_Scratch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])
Faults.shape

(1941, 34)

In [10]:
Faults = Faults.drop(columns=['TypeOfSteel_A400','TypeOfSteel_A300'])
Faults.shape

(1941, 32)

In [11]:
Faults.sample(3)

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scratch,Stains,Dirtiness,Bumps,Other_Faults
733,225,346,130372,130421,3228,166,89,260241,38,103,...,-0.5950,-0.3702,1.0000,0,0,1,0,0,0,0
1694,72,145,98042,98192,6156,233,211,543449,69,109,...,0.5133,-0.3103,1.0000,0,0,0,0,0,0,1
1847,265,276,183293,183331,227,22,38,32920,135,158,...,0.7105,0.1330,0.6839,0,0,0,0,0,0,1


In [12]:
from sklearn.neural_network import MLPClassifier

### Modeling

In [13]:
# Split data into X and y
features = Faults.columns[:-7]
labels = Faults.columns[-7:]
X = Faults[features]
y = Faults[labels]

In [14]:
from sklearn.model_selection import train_test_split 

# Train/test split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2,shuffle=True)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xtrain)

Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [16]:
m = MLPClassifier(hidden_layer_sizes=(5, 5, 5),
                 max_iter=10000) 
m.fit(Xtrain,ytrain)
m.score(Xtrain,ytrain)

0.4722938144329897

In [17]:
m = MLPClassifier(hidden_layer_sizes=(8, 8, 8),
                 max_iter=10000) 
m.fit(Xtrain,ytrain)
m.score(Xtrain,ytrain)

0.6527061855670103

In [23]:
def createmodel(n_layers, first_layer_nodes, last_layer_nodes, activation_func, loss_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=Xtrain.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer='adam', loss=loss_func, metrics = ["accuracy"]) #note: metrics could also be 'mse'
    
    return model

In [27]:
model =  KerasClassifier(build_fn=createmodel, verbose = False)  

In [28]:
activation_funcs = ['sigmoid', 'relu', 'tanh'] 
loss_funcs = ['binary_crossentropy','hinge']
param_grid = dict(n_layers=[2,3], first_layer_nodes = [64,32,16], last_layer_nodes = [4],  activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [100], epochs = [20,60])
grid = GridSearchCV(estimator = model, param_grid = param_grid)

In [30]:
print(grid.best_score_)
print(grid.best_params_)

nan
{'activation_func': 'sigmoid', 'batch_size': 100, 'epochs': 20, 'first_layer_nodes': 64, 'last_layer_nodes': 4, 'loss_func': 'binary_crossentropy', 'n_layers': 2}


In [31]:
m = MLPClassifier(hidden_layer_sizes=(64, 4), random_state=1, max_iter=500, warm_start=True)
for i in range(10):
    m.fit(Xtrain, ytrain)
    print(m.score(Xtrain,ytrain))

0.7461340206185567
0.8672680412371134
0.9072164948453608
0.9059278350515464
0.9001288659793815
0.8981958762886598
0.8936855670103093
0.8969072164948454
0.8930412371134021
0.8949742268041238


In [32]:
m = MLPClassifier(hidden_layer_sizes=(64, 4), random_state=1, max_iter=10000, warm_start=True)
for i in range(10):
    m.fit(Xtrain, ytrain)
    print(m.score(Xtrain,ytrain))

0.9265463917525774
0.9271907216494846
0.9233247422680413
0.9291237113402062
0.9252577319587629
0.9252577319587629
0.9239690721649485
0.9233247422680413
0.9239690721649485
0.9233247422680413


### Predictions

In [33]:
pred = m.predict(Xtest)
pred

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

### Accuracy from final model

In [34]:
from sklearn.metrics import classification_report
print(classification_report(ytest, pred))

              precision    recall  f1-score   support

           0       0.58      0.45      0.51        31
           1       0.81      0.74      0.77        34
           2       0.96      0.92      0.94        92
           3       0.91      0.77      0.83        13
           4       0.50      0.50      0.50        10
           5       0.51      0.59      0.55        71
           6       0.66      0.61      0.63       138

   micro avg       0.71      0.68      0.69       389
   macro avg       0.70      0.65      0.68       389
weighted avg       0.71      0.68      0.69       389
 samples avg       0.65      0.68      0.66       389

